In [4]:
import psycopg2
from psycopg2 import sql
import psycopg2.extras
from config import config

import pandas as pd
from pyarrow import feather

from typing import Iterator, Dict, Any

import subprocess

import urllib
import requests as r
import csv

In [5]:
conn = None
params = config()
conn = psycopg2.connect(**params)
conn.autocommit = True

In [3]:
def create_stations_table(cursor) -> None:
    cursor.execute("""
        DROP TABLE IF EXISTS stations;
        CREATE TABLE stations (
            code                TEXT NOT NULL,
            intersection        TEXT,
            latitude            DOUBLE PRECISION,
            longitude           DOUBLE PRECISION,
            city                TEXT NOT NULL,
            PRIMARY KEY(code, city)
        );
    """)

In [4]:
with conn.cursor() as cursor:
    create_stations_table(cursor)

In [25]:
def insert_execute_values(connection, all_stns) -> None:
    with connection.cursor() as cursor:
        create_stations_table(cursor)
        psycopg2.extras.execute_values(cursor, """
            INSERT INTO stations VALUES %s;
        """, ((
            stn[0],
            stn[1],
            stn[2],
            stn[3],
            stn[4]

        ) for stn in all_stns))

In [28]:
stations = feather.read_feather('feather-data/stations-all.feather')
stations.head()

,code,intersection,latitude,longitude,city
0,7000,King St W & Spadina Ave,43.250575,-79.833498,tor
1,7001,519 Church St,43.666552,-79.380943,tor
2,7002,51 Parliament St,43.651951,-79.362142,tor
3,7003,25 York St (ACC&Union Station South),43.643432,-79.381214,tor
4,7004,Liberty St & Fraser Ave Green P,43.637585,-79.424121,tor


In [29]:
stations.dtypes

code             object
intersection     object
latitude        float64
longitude       float64
city             object
dtype: object

In [33]:
stations_list = [tuple(x) for x in stations.values] # convert dataframe to tuples. Faster than converting to dict using to_dict

In [32]:
insert_execute_values(conn, tuples)

In [33]:
def create_trips_list(f):
    df = feather.read_feather(f)
    trips_list = (tuple(x) for x in df.values) #generator?
    
    return trips_list

In [10]:
def create_trips_table(cursor) -> None:
    cursor.execute("""
        DROP TABLE IF EXISTS trips;
        CREATE TABLE trips (
            trip_start_time         TIMESTAMP,
            start_station_code      TEXT,
            trip_stop_time          TIMESTAMP,
            end_station_code        TEXT,
            trip_duration_seconds   INT,
            membership_type         TEXT,
            city                    TEXT
        );
    """) 

In [13]:
def insert_trips_ev(cursor, trips) -> None:
        psycopg2.extras.execute_values(cursor, """
            INSERT INTO trips VALUES %s;
        """, ((
            trip[0],
            trip[1],
            trip[2],
            trip[3],
            trip[4],
            trip[5],
            trip[6]
        ) for trip in trips))

In [9]:
with conn.cursor() as cursor:
    create_trips_table(cursor)

In [29]:
def insert_city_trips(fnames):
    with conn.cursor() as cursor:
        for f in fnames:
            print(f)
            trips = create_trips_list(f)
            insert_trips_ev(cursor, trips)

In [36]:
tor_files = subprocess.getoutput('ls feather-data/t*.feather').split('\n')
tor_files[1:]

['feather-data/toronto_2016Q3.feather',
 'feather-data/toronto_2016Q4.feather',
 'feather-data/toronto_2017Q1.feather',
 'feather-data/toronto_2017Q2.feather',
 'feather-data/toronto_2017Q3.feather',
 'feather-data/toronto_2017Q4.feather']

In [63]:
insert_city_trips(tor_files[1:])

feather-data/toronto_2016Q3.feather
feather-data/toronto_2016Q4.feather
feather-data/toronto_2017Q1.feather
feather-data/toronto_2017Q2.feather
feather-data/toronto_2017Q3.feather
feather-data/toronto_2017Q4.feather


In [74]:
mtl_files = subprocess.getoutput('ls feather-data/m*.feather').split('\n')

In [80]:
insert_city_trips(mtl_files[:-6])

feather-data/montreal_2014-04.feather
feather-data/montreal_2014-05.feather
feather-data/montreal_2014-06.feather
feather-data/montreal_2014-07.feather
feather-data/montreal_2014-08.feather
feather-data/montreal_2014-09.feather
feather-data/montreal_2014-10.feather
feather-data/montreal_2014-11.feather
feather-data/montreal_2015-04.feather
feather-data/montreal_2015-05.feather
feather-data/montreal_2015-06.feather
feather-data/montreal_2015-07.feather
feather-data/montreal_2015-08.feather
feather-data/montreal_2015-09.feather
feather-data/montreal_2015-10.feather
feather-data/montreal_2015-11.feather
feather-data/montreal_2016-04.feather
feather-data/montreal_2016-05.feather
feather-data/montreal_2016-06.feather
feather-data/montreal_2016-07.feather
feather-data/montreal_2016-08.feather
feather-data/montreal_2016-09.feather
feather-data/montreal_2016-10.feather
feather-data/montreal_2016-11.feather
feather-data/montreal_2017-04.feather
feather-data/montreal_2017-05.feather
feather-data

In [81]:
van_files = subprocess.getoutput('ls feather-data/v*.feather').split('\n')

In [85]:
insert_city_trips(van_files[1:])

feather-data/vancouver_2017.feather
feather-data/vancouver_2018-01.feather
feather-data/vancouver_2018-02.feather
feather-data/vancouver_2018-03.feather
feather-data/vancouver_2018-04.feather
feather-data/vancouver_2018-05.feather
feather-data/vancouver_2018-06.feather
feather-data/vancouver_2018-07.feather
feather-data/vancouver_2018-08.feather
feather-data/vancouver_2018-09.feather
feather-data/vancouver_2018-10.feather
feather-data/vancouver_2018-11.feather
feather-data/vancouver_2018-12.feather
feather-data/vancouver_2019-01.feather
feather-data/vancouver_2019-02.feather
feather-data/vancouver_2019-03.feather


In [86]:
query = """ALTER TABLE trips ADD COLUMN trip_id SERIAL PRIMARY KEY;"""
with conn.cursor() as cursor:
    cursor.execute(query)

In [91]:
query = """ALTER TABLE trips ADD CONSTRAINT stations_fk FOREIGN KEY (start_station_code, city) REFERENCES stations (code, city);"""
with conn.cursor() as cursor:
    cursor.execute(query)

In [93]:
def get_weather_data(stationid, year, month):
    
    base_url = 'http://climate.weather.gc.ca/climate_data/bulk_data_e.html?'

    params = {
    'format': 'csv',
    'stationID': stationid,
    'Year': year,
    'Month': month, 
    'timeframe': 1,
    'submit': 'Download Data'}
    
    url = f'{base_url}{urllib.parse.urlencode(params)}'
    
    with r.Session() as session: # why use session?
        response = session.get(url)
        response.raise_for_status()

        decoded_response = response.content.decode('utf-8-sig') #https://stackoverflow.com/questions/17912307/u-ufeff-in-python-string
        data = decoded_response[decoded_response.index('"Date/'):].splitlines() #finds header row
        
        reader = csv.DictReader(data[1:], fieldnames = new_colnames)
        for record in reader:
            yield record # not really necessary here but useful if there were a large number of records

In [94]:
a = """ ()%Â°/""" #characters to replace
b = "_", "", "","perc","","deg_","_" #replacements (match on index)

orig_colnames = ['Date/Time', 'Year', 'Month', 'Day', 'Time', 'Temp (Â°C)', 'Temp Flag',
       'Dew Point Temp (Â°C)', 'Dew Point Temp Flag', 'Rel Hum (%)',
       'Rel Hum Flag', 'Wind Dir (10s deg)', 'Wind Dir Flag',
       'Wind Spd (km/h)', 'Wind Spd Flag', 'Visibility (km)',
       'Visibility Flag', 'Stn Press (kPa)', 'Stn Press Flag', 'Hmdx',
       'Hmdx Flag', 'Wind Chill', 'Wind Chill Flag', 'Weather']

trans_names = [col_name.translate(str.maketrans(dict(zip(a,b)))) for col_name in orig_colnames] #translation table
new_colnames = list(map(str.lower, trans_names))
new_colnames

['date_time',
 'year',
 'month',
 'day',
 'time',
 'temp_deg_c',
 'temp_flag',
 'dew_point_temp_deg_c',
 'dew_point_temp_flag',
 'rel_hum_perc',
 'rel_hum_flag',
 'wind_dir_10s_deg',
 'wind_dir_flag',
 'wind_spd_km_h',
 'wind_spd_flag',
 'visibility_km',
 'visibility_flag',
 'stn_press_kpa',
 'stn_press_flag',
 'hmdx',
 'hmdx_flag',
 'wind_chill',
 'wind_chill_flag',
 'weather']

In [95]:
mtl_stationid = 30165 #Trudeau Intl
tor_stationid = 48549 #Toronto City Ctr
van_stationid = 51442 #Vancouver Intl Airport

In [96]:
def create_weather_table(cursor) -> None:
    cursor.execute("""
        DROP TABLE IF EXISTS weather;
        CREATE TABLE weather (
            date_time               TIMESTAMP,
            temp_deg_c              REAL,
            rel_hum_perc            REAL,
            wind_spd_km_h           REAL,
            visibility_km           REAL,
            wind_chill              REAL,
            weather                 TEXT,
            city                    TEXT
        );
    """) 

In [97]:
def insert_weather(cursor, weather_data:Iterator[Dict[str, Any]], city) -> None:
        psycopg2.extras.execute_values(cursor, """
            INSERT INTO weather VALUES %s;
        """, ((
            w['date_time'],
            w['temp_deg_c'],
            w['rel_hum_perc'],
            w['wind_spd_km_h'],
            w['visibility_km'],
            w['wind_chill'],
            w['weather'],
            city
        ) for w in weather_data))

In [98]:
def clean_weather_data(gen_weather):
    weather_data = []
    for rec in gen_weather:
        rec.update((k, None) for k,v in rec.items() if not v or v == 'NA') 
        weather_data.append(rec)
    return weather_data #list of dicts

In [99]:
with conn.cursor() as cursor:
    create_weather_table(cursor)

In [100]:
from time import sleep

city = 'mtl'

with conn.cursor() as cursor:
    for year in [2014, 2015, 2016, 2017, 2018]:
        for month in [4, 5, 6, 7, 8, 9, 10, 11]:
            print(year, month)
            weather_data = clean_weather_data(get_weather_data(stationid = mtl_stationid, year = year, month = month))
            insert_weather(cursor, weather_data, city)
            sleep(1) #time delay for api call

2014 4
2014 5
2014 6
2014 7
2014 8
2014 9
2014 10
2014 11
2015 4
2015 5
2015 6
2015 7
2015 8
2015 9
2015 10
2015 11
2016 4
2016 5
2016 6
2016 7
2016 8
2016 9
2016 10
2016 11
2017 4
2017 5
2017 6
2017 7
2017 8
2017 9
2017 10
2017 11
2018 4
2018 5
2018 6
2018 7
2018 8
2018 9
2018 10
2018 11


In [101]:
def insert_city_weather(stationid, year, startmth, endmth, city):
    
    with conn.cursor() as cursor:
        for month in range(startmth, endmth+1):
            print(year, month)
            weather_data = clean_weather_data(get_weather_data(stationid = stationid, year = year, month = month))
            insert_weather(cursor, weather_data, city)
            sleep(1) #time delay for api call

In [105]:
insert_city_weather(tor_stationid, 2017, 1, 12, 'tor')

2017 1
2017 2
2017 3
2017 4
2017 5
2017 6
2017 7
2017 8
2017 9
2017 10
2017 11
2017 12


In [106]:
insert_city_weather(tor_stationid, 2016, 7, 12, 'tor')

2016 7
2016 8
2016 9
2016 10
2016 11
2016 12


In [107]:
insert_city_weather(van_stationid, 2017, 1, 12, 'van')

2017 1
2017 2
2017 3
2017 4
2017 5
2017 6
2017 7
2017 8
2017 9
2017 10
2017 11
2017 12


In [108]:
insert_city_weather(van_stationid, 2018, 1, 12, 'van')

2018 1
2018 2
2018 3
2018 4
2018 5
2018 6
2018 7
2018 8
2018 9
2018 10
2018 11
2018 12


In [109]:
insert_city_weather(van_stationid, 2019, 1, 3, 'van')

2019 1
2019 2
2019 3


In [110]:
query = """ALTER TABLE weather ADD PRIMARY KEY (date_time, city);"""
with conn.cursor() as cursor:
    cursor.execute(query)

In [111]:
query = """ALTER TABLE trips ADD COLUMN trip_start_time_hr TIMESTAMP;"""
with conn.cursor() as cursor:
    cursor.execute(query)

In [112]:
query = """UPDATE trips SET trip_start_time_hr = date_trunc('hour', trip_start_time);"""
with conn.cursor() as cursor:
    cursor.execute(query)

In [113]:
query = """ALTER TABLE trips ADD CONSTRAINT weather_fk FOREIGN KEY (trip_start_time_hr, city) REFERENCES weather (date_time, city);"""
with conn.cursor() as cursor:
    cursor.execute(query)

In [114]:
# remove extra days for montreal in weather table

In [ ]:
conn.close()